In [1]:
import os
import pandas as pd
import pickle
import numpy as np
import warnings
import seaborn as sns
import re
from names_dataset import NameDataset
import time
import datetime 
import import_data
import matplotlib.pyplot as plt
import gender_classification
import calculate_AR
import ttest
import pandas as pd
import numpy as np
import time
import datetime 
import LinearReg
from sklearn.linear_model import LinearRegression

from IPython.display import clear_output

path = "/Users/admin/Documents/Interd Project/data/"

In [2]:
df = pd.read_pickle(path +"df_incl_gender.pkl")
df = df[df["Gender"]!="na"]

DR_companies = pd.read_pickle(path + "DR_companies.pkl")
DR_market = pd.read_pickle(path + "DR_market.pkl")

isin = pd.read_excel(path+ 'ISIN_merge.xlsx') 
df = df.merge(isin,left_on="Ticker", right_on="TICKER SYMBOL", how='left').drop(columns="TICKER SYMBOL")

not_found = df[df["ISIN CODE"].isna()]
print("Transactions without ISIN Code: " +str(len(not_found))+ ". TICKERS: " + str(not_found["Ticker"].unique()))
df.dropna(subset=['ISIN CODE'], inplace=True)

df_isins = df["ISIN CODE"].tolist()
dr_isins = DR_companies.index.values.tolist()

isin_not_in_dr = set(df_isins)-set(dr_isins)
print(len(isin_not_in_dr))
pd.DataFrame(isin_not_in_dr).to_excel(path+"ISIN_noinDR.xlsx")
df = df[~df['ISIN CODE'].isin(isin_not_in_dr)]
print("DF shape after dropping companies, for which we don't have daily returns: " + str(df.shape))

df["Filing Date"] = df['Filing Date'].apply(lambda x: pd.to_datetime(x).normalize())
DD_sameDay_samePerson = df[df.duplicated(subset=['Filing Date', 'Insider Name','Ticker','Trade Type'],keep=False)].to_excel(path + "DD_sameDayPerson.xlsx")
df.drop_duplicates(subset=['Filing Date', 'Insider Name','Ticker','Trade Type'], keep='first',inplace=True)
print(df.shape)

# some more preprocessing before calculating the individual Abnormal Returns
# we need to drop all transactions that are too recent, wo we cannot calculate event window
print("Number of transactions after June 23 (too recent): " + str(len(df[df['Filing Date'] > "2023-06-01 00:00:00"])))
df[df['Filing Date'] > "2023-06-01 00:00:00"].to_excel(path+"FilingDate_toorecent.xlsx")
df = df.drop(df[df['Filing Date'] > "2023-06-01 00:00:00"].index)

#infinite values cannot be processed, we replace them with nan values
DR_companies.replace([np.inf, -np.inf], np.nan, inplace=True)

Transactions without ISIN Code: 6. TICKERS: ['PBBI']
121
DF shape after dropping companies, for which we don't have daily returns: (589698, 9)
(571763, 9)
Number of transactions after June 23 (too recent): 8060


In [78]:
date_notin_dr = 0 

def calc_windows(event_day,dr,L1,L2):
    
    if event_day not in dr:
        return pd.DataFrame(np.zeros((1, 200))).astype("float32"),pd.DataFrame(np.zeros((1, 41))).astype("float32")
        print(event_day)
        date_notin_dr+=1
    #same index for market and company
    event_index = dr.columns.get_loc(event_day)

    T1_index = (event_index-1)-L2
    T2_index = (event_index)+L2
    T0_index = T1_index-L1
    T3_index = T2_index+L1
    #print(event_day,T0_index, T1_index,T2_index,T3_index) 

    
    # Estimation window
    estimation_window = dr.iloc[:,T0_index:T1_index]
    estimation_window.transpose().dropna(thresh=100,inplace=True)
    
    #estimation_window.transpose().fillna(method="bfill",inplace=True)
    #estimation_window.transpose().fillna(method="ffill",inplace=True)
    #print(estimation_window.shape)

    # Event-Window
    event_window = dr.iloc[:,T1_index:T2_index]
    #print(event_window.shape)
    
    return estimation_window,event_window


def calc_linreg(X_train,y_train,X_test,y_test):
    xmean= np.mean(X_train)
    ymean= np.mean(y_train)
    X_train = np.c_[np.ones(X_train.shape[0]), X_train]
    #print(X_train)
    model = LinearRegression(fit_intercept=False)
    
    model.fit(X_train,y_train)
    
    X_test = np.c_[np.ones(X_test.shape[0]), X_test]
    y_pred = model.predict(X_test)
    
    b = model.coef_[1]
    residuals = (y_test-y_pred)
    a = (xmean)-(b*ymean)

    return a,b,residuals

def gunnar_run(X,Y):
    
    # add a constant to the X matrix
    X = np.c_[np.ones(X.shape[0]), X]

    # calculate the coefficients
    beta = np.linalg.inv(X.T @ X) @ X.T @ Y

    # calculate the residuals
    eps = Y - X @ beta

    return beta[0], beta[1], eps

def run_calculation_AR(date,cr,mr, L1 = 200, L2 = 20):
    
    cr = pd.DataFrame(cr).transpose()
    
    est_window_market,event_window_market = calc_windows(date,mr,L1,L2)
    est_window_comp,event_window_comp = calc_windows(date,cr,L1,L2)
    return est_window_comp,event_window_comp 

    if (est_window_market.isnull().values.all() or est_window_comp.isnull().values.any()):
        print(est_window_market.iloc[:,-1:])
        print(est_window_comp.iloc[:,-1:])
        return pd.DataFrame(np.zeros((1, 41))).astype("float32")
    
    #return est_window_market,event_window_market,est_window_comp,event_window_comp

    a,b,eps = calc_linreg(est_window_market.values[0,:],est_window_comp.values[0,:],event_window_market.values[0,:],event_window_comp.values[0,:])
    #a,b,eps = gunnar_run(est_window_market.values[0,:],est_window_comp.values[0,:])
    #coefs,interc,residuals = calc_linreg(est_window_market.values[0,:],est_window_comp.values[0,:])
    AR = event_window_comp.values - a - (b*event_window_market.values)
    #print(coefs,interc,residuals)
   
    return AR,eps
    

In [79]:
mr = DR_market[:1]
cols = [str(item)for item in range(-20,21)]
start_time = time.time()
epses = []
empty = pd.DataFrame(np.zeros((1, 41))).astype("float32")
empty_all = []


ARs = pd.DataFrame()
    
count = 0

for v in df.values[:100,:]:

    event_date = v[0]
    isin = v[-1]

    cr = DR_companies.loc[isin]

    #abnormal_returns,eps = run_calculation_AR(event_date,cr,mr)
    est_c,ev_c = run_calculation_AR(event_date,cr,mr)
    check_if_empty = (abnormal_returns == empty).all()
    if (check_if_empty == True).all():
        empty_all.append(abnormal_returns)
        continue

    ARs = pd.concat([ARs,pd.DataFrame(abnormal_returns)])
    epses.append(eps)
    count+=1





In [84]:
if not (est_c.transpose().isna().sum().all()>100):
    print ("True")

True
